In [1]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy  as np
from sqlalchemy import create_engine
import logging
import re

In [2]:
MYSQL_HOST = 
MYSQL_USER = 
MYSQL_PASSWORD = 
MYSQL_CHARSET = 
wind = create_engine()

In [3]:
def set_logger(log_file=None, level=logging.DEBUG, mode='a', name=""):
    """
    written by Sha
    :return:
    """
    formatter = logging.Formatter(fmt='[%(asctime)s] %(levelname)s: %(module)s: %(message)s')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)
    if log_file is not None:
        handler_file = logging.FileHandler(filename=log_file, encoding='utf-8', mode=mode)
        handler_file.setFormatter(formatter)
        logger.addHandler(handler_file)
    return logger

In [4]:
def write_log(logger, content):
    if logger is not None:
        logger.info(content)
    else:
        print(content)

In [5]:
def split_date(start_date):
    """
    :param start_date: could be none
    :return: [(s, e), (s, e), (s, None)]
    """
    if start_date is None:
        return [(None, None)]
    start_dt = datetime.strptime(str(start_date), '%Y%m%d')
    today = datetime.today()
    periods = []
    _start = start_dt
    _end = datetime(start_dt.year, 12, 31)
    for i in range(100):
        if _end.date() < today.date():
            periods.append((int(_start.strftime('%Y%m%d')), int(_end.strftime('%Y%m%d'))))
        else:
            periods.append((int(_start.strftime('%Y%m%d')), int(today.strftime("%Y%m%d"))))
            break
        _temp = _end + relativedelta(days=1)
        _end = _end + relativedelta(years=1)
        _start = _temp
    return periods

In [6]:
stock_info_sql = "SELECT * FROM ASHAREDESCRIPTION WHERE S_INFO_WINDCODE NOT LIKE 'A%'"

In [7]:
df_stock_info = pd.read_sql_query(stock_info_sql, wind)
df_stock_info.head()

,OBJECT_ID,S_INFO_WINDCODE,S_INFO_CODE,S_INFO_NAME,S_INFO_COMPNAME,S_INFO_COMPNAMEENG,S_INFO_ISINCODE,S_INFO_EXCHMARKET,S_INFO_LISTBOARD,S_INFO_LISTDATE,S_INFO_DELISTDATE,S_INFO_SEDOLCODE,CRNCY_CODE,S_INFO_PINYIN,S_INFO_LISTBOARDNAME,IS_SHSC,S_INFO_COMPCODE,OPDATE,OPMODE
0,{0000B5D4-B786-4B03-A7B0-50E3767ABF84},600373.SH,600373,中文传媒,中文天地出版传媒集团股份有限公司,Chinese Universe Publishing and Media Group Co...,CNE0000019X4,SSE,434004000,20020304,None,None,CNY,zwcm,主板,1.0,1600373,2015-09-09 09:07:23,0
1,{001EE265-3F28-DD72-E050-C80A100105DE},300557.SZ,300557,理工光科,武汉理工光科股份有限公司,Wuhan Ligong Guangke Co.Ltd.,CNE100002DC1,SZSE,434001000,20161101,None,None,CNY,lggk,创业板,0.0,1uF6BE2FE5,2017-01-05 15:26:58,0
2,{0058F562-B872-4558-A53B-313201BF5C86},000416.SZ,000416,民生控股,民生控股股份有限公司,"Minsheng Holdings Co.,Ltd",CNE0000009P1,SZSE,434004000,19960719,None,None,CNY,mskg,主板,0.0,1000416,2015-09-09 09:08:04,0
3,{00910667-25C7-429A-AF39-515A56E11D9E},002156.SZ,002156,通富微电,通富微电子股份有限公司,"TongFu Microelectronics Co.,Ltd.",CNE1000006C3,SZSE,434003000,20070816,None,None,CNY,tfwd,中小企业板,2.0,2o557A92BF,2017-03-20 00:22:43,0
4,{00A97A43-6466-41D6-B7B7-017B6868D630},600500.SH,600500,中化国际,中化国际(控股)股份有限公司,Sinochem International Corporation,CNE0000011R3,SSE,434004000,20000301,None,None,CNY,zhgj,主板,1.0,1600500,2015-09-09 09:08:19,0


In [8]:
def get_SuperTable(start_date=20060101, end_date=None):
    SuperTable_sql = "SELECT *" + \
                     " FROM ASHARESUPERTABLE" + \
                     " WHERE TRADE_DT > '" + str(start_date) + \
                     "' AND TRADE_DT <= '" + str(end_date) + \
                     "' ORDER BY TRADE_DT"
    df = pd.read_sql_query(SuperTable_sql, wind)
    return df

In [9]:
def get_DataDate(start_date=20060101, end_date=None):
    DataDate_sql = "SELECT *" + \
                " FROM ASHARECALENDAR" + \
                " WHERE S_INFO_EXCHMARKET = 'SSE' AND TRADE_DAYS > '" + str(start_date) + \
                "' AND TRADE_DAYS <= '" + str(end_date) + \
                "' ORDER BY TRADE_DAYS"
    df = pd.read_sql_query(DataDate_sql, wind)
    return df

In [10]:
class datachecking:
    
    def __init__(self,data_df,stock_info,DataDate,logger,start_date=20060101,end_date=None):
        '''
        data_df :
            index  : sid DataDate (ticktime)
            columns: price,derivatives,INDUSTRY,UNIVERSE,ST,.....
            
        stock_info : sid,list_date,delist_date
        DataDate   : trading calendar    
        '''
        self.data_df = data_df
        self.stock_info = stock_info
        self.DataDate = DataDate
        self.start_date = start_date
        self.end_date = end_date
        self.logger = logger
        pass
    
    def check_abvalue(self):
        '''
        inf,nan,price==0
        '''
        if(self.data_df.empty == True):
            pass
        else:
            for sid, data in self.data_df.groupby(level = 'S_INFO_WINDCODE'):
                print(sid + "check abvalue")
                for i in range(len(data)):
                    for j in range(len(data.columns)):
                        if((data.iloc[i, j] == None)):
                            print(str(data.index[i]) + " " + data.columns[j] + "does not have data")
                            write_log(self.logger, str(data.index[i]) + " " + data.columns[j] + "does not have data")
                        elif((data.iloc[i, j] > 1e15) | (data.iloc[i, j] < -1e15)):
                            print(str(data.index[i]) + " " + data.columns[j] + " have infinity data")
                            write_log(self.logger, str(data.index[i]) + " " + data.columns[j] + " have infinity data")
                        elif((data.iloc[i, j] == 0) & (re.search('S_DQ', data.columns[j]) != None)):
                            print(str(data.index[i]) + " " + data.columns[j] + " have zero data")
                            write_log(self.logger, str(data.index[i]) + " " + data.columns[j] + " have zero data")
                        else:
                            pass
                    
        
        for i in range(len(self.stock_info)):
            if(self.stock_info.loc[i, 'S_INFO_LISTDATE'] == None):
                print("processing " + self.stock_info.loc[i, 'S_INFO_WINDCODE'] + " does not have list date")
                write_log(self.logger, self.stock_info.loc[i, 'S_INFO_WINDCODE'] + " does not have list date")
            else:
                pass
        pass
    
    def check_datadate(self):
        '''
        unique DataDate from data_df match the trading calendar during start_date and end_date
        '''
        print("check data date")
        if(self.data_df.empty == True):
            pass
        else:
            new_df = self.data_df.reset_index()
            st = set(new_df['TRADE_DT'])
            result = [self.DataDate.loc[i, 'TRADE_DAYS'] for i, e in enumerate(self.DataDate['TRADE_DAYS']) if e not in st]
            if result:
                print("Stock trade days cannot match trade calendar " + str(result))
                write_log(self.logger,"Stock trade days cannot match trade calendar " + str(result))
        
        pass
    
    def check_idx_unique(self):
        '''
        columns name unique
        sid DataDate (tiktime) unique
        '''
        print("check idx unique")
        if(self.data_df.empty == True):
            pass
        else:
            for i in range(len(self.data_df)):
                if(self.data_df.index.duplicated()[i] == True):
                    print(str(self.data_df.index[i]) + " have duplicate sid and DataDate")
                    write_log(self.logger, str(self.data_df.index[i]) + " have duplicate sid and DataDate")
            for j in range(len(self.data_df.columns)):
                if(self.data_df.columns.duplicated()[j] == True):
                    print(str(self.data_df.columns[j]) + " have duplicate sid and DataDate")
                    write_log(self.logger, str(self.data_df.columns[j]) + " have duplicate sid and DataDate")
        pass
    
    def check_listed_range(self):
        '''
        for DataDate range of each sid
        max(DataDate)<delist_date
        min(DataDate)>list_date or start_date
        '''
        if(self.data_df.empty == True):
            pass
        else:
            for sid, data in self.data_df.groupby(level = 'S_INFO_WINDCODE'):
                print(sid + "check listed range")
                ListDate = self.stock_info.loc[self.stock_info['S_INFO_WINDCODE'] == sid, 'S_INFO_LISTDATE'].values
                DelistDate = self.stock_info.loc[self.stock_info['S_INFO_WINDCODE'] == sid, 'S_INFO_DELISTDATE'].values
                list_ = []
                for i in range(len(data)):
                    list_.append(data.iloc[:, 1].index[i][1])
                MaxDate = max(list_)
                MinDate = min(list_)
                if(int(MinDate) < int(ListDate)):
                    print(sid + " have earlier trade days " + MinDate + " earlier than list date " + ListDate)
                    write_log(self.logger, sid + " have earlier trade days " + MinDate + " earlier than list date " + ListDate)
                
                elif(DelistDate == None):
                    pass
                
                elif((int(MaxDate) > int(DelistDate)) & (DelistDate != None)):
                    print(sid + " have later trade days " + MaxDate + " later than delist date " + DelistDate)
                    write_log(self.logger, sid + " have later trade days " + MaxDate + " later than delist date " + DelistDate)
                    
        pass
    
    def check_sid_tidx(self):
        '''
        listed stock should have data at each DataDate (ticktime)
        '''
        if(self.data_df.empty == True):
            pass
        else:
            for sid, data in self.data_df.groupby(level = 'S_INFO_WINDCODE'):
                print(sid + "check sid tidx")
                list_ = []
                for i in range(len(data)):
                    list_.append(data.iloc[:, 1].index[i][1])
                st = set(list_)
                ListDate = self.stock_info.loc[self.stock_info['S_INFO_WINDCODE'] == sid, 'S_INFO_LISTDATE'].values
                DelistDate = self.stock_info.loc[self.stock_info['S_INFO_WINDCODE'] == sid, 'S_INFO_DELISTDATE'].values
                if(DelistDate == None):
                    condition = self.DataDate['TRADE_DAYS'].apply(lambda x: int(x) >= int(ListDate))
                elif(DelistDate != None):
                    condition = self.DataDate['TRADE_DAYS'].apply(lambda x: (int(x) >= int(ListDate)) & (int(x) < int(DelistDate)))
                new_DataDate = self.DataDate.loc[condition]
                result = [new_DataDate.loc[i, 'TRADE_DAYS'] for i, e in enumerate(new_DataDate['TRADE_DAYS']) if e not in st]
                if result:
                    print(sid + " did not trade on " + str(result))
                    write_log(self.logger, sid + " did not trade on " + str(result))
        pass
    
    def check_price(self):
        '''
        (adj)low <=...<=adj(high)
        '''
        if(self.data_df.empty == True):
            pass
        else:
            print("check price")
            LOW_HIGH = self.data_df.loc[self.data_df['S_DQ_LOW'] >  self.data_df['S_DQ_HIGH'],].index
            LOW_OPEN = self.data_df.loc[self.data_df['S_DQ_LOW'] >  self.data_df['S_DQ_OPEN'],].index
            LOW_CLOSE = self.data_df.loc[self.data_df['S_DQ_LOW'] >  self.data_df['S_DQ_CLOSE'],].index
            OPEN_HIGH = self.data_df.loc[self.data_df['S_DQ_OPEN'] >  self.data_df['S_DQ_HIGH'],].index
            CLOSE_HIGH = self.data_df.loc[self.data_df['S_DQ_CLOSE'] >  self.data_df['S_DQ_HIGH'],].index
            ADJLOW_ADJHIGH = self.data_df.loc[self.data_df['S_DQ_ADJLOW'] >  self.data_df['S_DQ_ADJHIGH'],].index
            ADJLOW_ADJOPEN = self.data_df.loc[self.data_df['S_DQ_ADJLOW'] >  self.data_df['S_DQ_ADJOPEN'],].index
            ADJLOW_ADJCLOSE = self.data_df.loc[self.data_df['S_DQ_ADJLOW'] >  self.data_df['S_DQ_ADJCLOSE'],].index
            ADJOPEN_ADJHIGH = self.data_df.loc[self.data_df['S_DQ_ADJOPEN'] >  self.data_df['S_DQ_ADJHIGH'],].index
            ADJCLOSE_ADJHIGH = self.data_df.loc[self.data_df['S_DQ_ADJCLOSE'] >  self.data_df['S_DQ_ADJHIGH'],].index
            if ((len(LOW_HIGH)==0) &(len(LOW_OPEN)==0) & (len(LOW_CLOSE)==0) & (len(OPEN_HIGH)==0) & 
                (len(ADJLOW_ADJHIGH)==0) & (len(CLOSE_HIGH)==0) & (len(ADJLOW_ADJOPEN)==0) & 
                (len(ADJLOW_ADJCLOSE)==0) & (len(ADJOPEN_ADJHIGH)==0) & (len(ADJCLOSE_ADJHIGH)==0)):
                pass
            else:
                WrongPrice = []
                for i in range(len(LOW_HIGH)): WrongPrice.append(LOW_HIGH[i])
                for i in range(len(LOW_OPEN)): WrongPrice.append(LOW_OPEN[i])
                for i in range(len(LOW_CLOSE)): WrongPrice.append(LOW_CLOSE[i])
                for i in range(len(OPEN_HIGH)): WrongPrice.append(OPEN_HIGH[i])
                for i in range(len(CLOSE_HIGH)): WrongPrice.append(CLOSE_HIGH[i])
                for i in range(len(ADJLOW_ADJHIGH)): WrongPrice.append(ADJLOW_ADJHIGH[i])
                for i in range(len(ADJLOW_ADJOPEN)): WrongPrice.append(ADJLOW_ADJOPEN[i])
                for i in range(len(ADJLOW_ADJCLOSE)): WrongPrice.append(ADJLOW_ADJCLOSE[i])
                for i in range(len(ADJOPEN_ADJHIGH)): WrongPrice.append(ADJOPEN_ADJHIGH[i])
                for i in range(len(ADJCLOSE_ADJHIGH)): WrongPrice.append(ADJCLOSE_ADJHIGH[i])
                print(str(WrongPrice) + " have wrong price")
                write_log(self.logger, str(WrongPrice) + " have wrong price")
        pass
        '''
        else:
            for sid, data in self.data_df.groupby(level = 'S_INFO_WINDCODE'):
                print(sid + "check price")
                for i in range(len(data)):
                    Low = data.iloc[i]['S_DQ_LOW']
                    Open = data.iloc[i]['S_DQ_OPEN']
                    High = data.iloc[i]['S_DQ_HIGH']
                    Close = data.iloc[i]['S_DQ_CLOSE']
                    AdjLow = data.iloc[i]['S_DQ_ADJLOW']
                    AdjOpen = data.iloc[i]['S_DQ_ADJOPEN']
                    AdjHigh = data.iloc[i]['S_DQ_ADJHIGH']
                    AdjClose = data.iloc[i]['S_DQ_ADJCLOSE']
                    if((Low <= High) & (Low <= Open) & (Low <= Close) & (Open <= High) & (Close <= High) & 
                       (AdjLow <= AdjHigh) & (AdjLow <= AdjOpen) & (AdjLow <= AdjClose) & (AdjOpen <= AdjHigh) & (AdjClose <= AdjHigh)):
                        pass
                    else:
                        print(str(data.index[i]) + " have wrong price")
                        write_log(self.logger, str(data.index[i]) + " have wrong price")
        
        pass
        '''
        
        
      
    def check_volume(self):
        '''
        if volume==0:
            ret==0
            open==high==low==close==pre_close, same for adj price
        '''
        if(self.data_df.empty == True):
            pass
        else:
            print("check volume")
            volume_list = df.loc[df['S_VOLUME'] ==0,]
            open_low = volume_list.loc[volume_list['S_DQ_LOW']!=volume_list['S_DQ_OPEN'],].index
            open_high = volume_list.loc[volume_list['S_DQ_HIGH']!=volume_list['S_DQ_OPEN'],].index
            open_close =  volume_list.loc[volume_list['S_DQ_OPEN']!=volume_list['S_DQ_CLOSE'],].index  
            open_pre = volume_list.loc[volume_list['S_DQ_OPEN']!=volume_list['S_DQ_PRECLOSE'],].index
            high_low = volume_list.loc[volume_list['S_DQ_HIGH']!=volume_list['S_DQ_LOW'],].index
            high_close =  volume_list.loc[volume_list['S_DQ_HIGH']!=volume_list['S_DQ_CLOSE'],].index 
            high_pre = volume_list.loc[volume_list['S_DQ_HIGH']!=volume_list['S_DQ_PRECLOSE'],].index
            low_pre = volume_list.loc[volume_list['S_DQ_LOW']!=volume_list['S_DQ_PRECLOSE'],].index
            low_close =  volume_list.loc[volume_list['S_DQ_LOW']!=volume_list['S_DQ_CLOSE'],].index 
            close_pre =  volume_list.loc[volume_list['S_DQ_PRECLOSE']!=volume_list['S_DQ_CLOSE'],].index 

            adj_open_low = volume_list.loc[volume_list['S_DQ_ADJLOW']!=volume_list['S_DQ_ADJOPEN'],].index
            adj_open_high = volume_list.loc[volume_list['S_DQ_ADJHIGH']!=volume_list['S_DQ_ADJOPEN'],].index
            adj_open_close =  volume_list.loc[volume_list['S_DQ_ADJOPEN']!=volume_list['S_DQ_ADJCLOSE'],].index  
            adj_high_low = volume_list.loc[volume_list['S_DQ_ADJHIGH']!=volume_list['S_DQ_ADJLOW'],].index
            adj_high_close =  volume_list.loc[volume_list['S_DQ_ADJHIGH']!=volume_list['S_DQ_ADJCLOSE'],].index 
            adj_low_close =  volume_list.loc[volume_list['S_DQ_ADJLOW']!=volume_list['S_DQ_ADJCLOSE'],].index 

            if (len(open_low)==0 & len(open_high)==0 & len(open_close)==0 & len(open_pre)==0 & 
                len(high_low==0) & len(high_close)==0 & len(high_pre==0) & len(low_pre)==0 & 
                len(low_close)==0 &len(close_pre)==0 & len(adj_open_low)==0 & len(adj_open_high)==0 &
                len(adj_open_close)==0 & len(adj_high_low)==0 & len(adj_high_close) == 0 & len(adj_low_close) ==0):
                pass
            else:
                WrongVolume = []
                for i in range(len(open_low)): WrongVolume.append(open_low[i])
                for i in range(len(open_high)): WrongVolume.append(open_high[i])
                for i in range(len(open_close)): WrongVolume.append(open_close[i])
                for i in range(len(open_pre)): WrongVolume.append(open_pre[i])
                for i in range(len(high_low)): WrongVolume.append(high_low[i])
                for i in range(len(high_close)): WrongVolume.append(high_close[i])
                for i in range(len(high_pre)): WrongVolume.append(high_pre[i])
                for i in range(len(low_pre)): WrongVolume.append(low_pre[i])
                for i in range(len(low_close)): WrongVolume.append(low_close[i])
                for i in range(len(close_pre)): WrongVolume.append(close_pre[i])
                    
                for i in range(len(adj_open_low)): WrongVolume.append(adj_open_low[i])
                for i in range(len(adj_open_high)): WrongVolume.append(adj_open_high[i])
                for i in range(len(adj_open_close)): WrongVolume.append(adj_open_close[i])
                for i in range(len(adj_high_low)): WrongVolume.append(adj_high_low[i])
                for i in range(len(adj_high_close)): WrongVolume.append(adj_high_close[i])
                for i in range(len(adj_low_close)): WrongVolume.append(adj_low_close[i])
                
                print(str(WrongVolume) + " does not have equal price/adjprice")
                write_log(self.logger, str(WrongVolume) + " does not have equal price/adjprice")
        pass    
        '''
        else:
            for sid, data in self.data_df.groupby(level = 'S_INFO_WINDCODE'):
                print(sid + "check volume")
                for i in range(len(data)):
                    if(data.iloc[i]['S_VOLUME'] == 0):
                        price = data.iloc[i]['S_DQ_OPEN']
                        adjprice = data.iloc[i]['S_DQ_ADJOPEN']
                        if((data.iloc[i]['S_DQ_LOW'] == price) & (data.iloc[i]['S_DQ_HIGH'] == price) & 
                           (data.iloc[i]['S_DQ_CLOSE'] == price) & (data.iloc[i]['S_DQ_ADJLOW'] == adjprice) & 
                           (data.iloc[i]['S_DQ_ADJHIGH'] == adjprice) & (data.iloc[i]['S_DQ_ADJCLOSE'] == adjprice) &
                           (data.iloc[i]['S_PCTCHANGE'] == 0)):
                            pass
                        else:
                            print(str(data.index[i]) + " does not have equal price/adjprice")
                            write_log(self.logger, str(data.index[i]) + " does not have equal price/adjprice")
        
        pass
        '''
    
    def list_ret_dist(self,lower_bound=-0.1,upper_bound=0.1):
        '''
        list the return of which sid out of the bound
        '''
        if(self.data_df.empty == True):
            pass
        else:
            print("check list ret dist")
            upper = self.data_df.loc[self.data_df['S_PCTCHANGE'] >= upper_bound * 100,].index
            lower = self.data_df.loc[self.data_df['S_PCTCHANGE'] <= lower_bound * 100,].index
            
            if((len(upper) + len(lower)) > 0):
                LimitPrice = []
                for i in range(len(upper)): LimitPrice.append(upper[i])
                for i in range(len(lower)): LimitPrice.append(lower[i])
                    
                print(str(LimitPrice) + " have reach upper bound or lower bound")
                write_log(self.logger, str(LimitPrice) + " have reach upper bound or lower bound")
        pass
        '''
        else:
            for sid, data in self.data_df.groupby(level = 'S_INFO_WINDCODE'):
                print(sid + "check list ret dist")
                for i in range(len(data)):
                    if((data.iloc[i]['S_PCTCHANGE'] >= upper_bound * 100) | (data.iloc[i]['S_PCTCHANGE'] <= lower_bound * 100)):
                        print(str(data.index[i]) + " have reach upper bound or lower bound")
                        write_log(self.logger, str(data.index[i]) + " have reach upper bound or lower bound")
        pass
        '''
        
        
        
    
    def list_st_change(self):
        '''
        which become ST or become non-ST
        '''
        if(self.data_df.empty == True):
            pass
        else:
            for sid, data in self.data_df.groupby(level = 'S_INFO_WINDCODE'):
                print(sid + "check list st change")
                for i in range(len(data)):
                    if((data.iloc[i -1]['ST_FLAG'] == 0) & (data.iloc[i]['ST_FLAG'] == 1) & (i > 0)):
                        print(str(data.index[i]) + " from non-ST stock become ST stock")
                        write_log(self.logger, str(data.index[i]) + " from non-ST stock become ST stock")
                    elif((data.iloc[i - 1]['ST_FLAG'] == 1) & (data.iloc[i]['ST_FLAG'] == 0) & (i > 0)):
                        print(str(data.index[i]) + " from ST stock become non-ST stock")
                        write_log(self.logger, str(data.index[i]) + " from ST stock become non-ST stock")
        pass

In [1]:
if __name__ == '__main__':
    #print([i for i in split_date(20061101)])
    #print([i for i in split_date(20181228)])
    date_list = [str((datetime.today() - timedelta(days = 30)).strftime('%Y%m%d')), 
                 str(datetime.today().strftime('%Y%m%d'))]
    #date_list = [20180901, 20190101]
    for i in range(len(date_list)-1):
        start = datetime.now()
        df = get_SuperTable(date_list[i], date_list[i+1])
        df = df.set_index(['S_INFO_WINDCODE', 'TRADE_DT'])
        print("读取数据时间： " + str(datetime.now() - start))
        df_DataDate = get_DataDate(date_list[i], date_list[i+1])
        logger_file = 'log' + str(date_list[i]) + '-' + str(date_list[i+1]) + '_Follower'
        wind_logger = set_logger(log_file=logger_file)
        wind_logger.info("This is a test")
        wind_logger.debug("This is a test")
        a = datachecking(df, df_stock_info, df_DataDate, wind_logger, 20191201, str(datetime.today().strftime('%Y%m%d')))
        start = datetime.now()
        a.check_abvalue()
        a.check_datadate()
        #a.check_idx_unique()
        a.check_listed_range()
        a.check_sid_tidx()
        a.check_price()
        a.check_volume()
        a.list_ret_dist()
        a.list_st_change()
        print("筛查数据所需时间： " + str(datetime.now() - start))

NameError: name 'datetime' is not defined